## Notebook component to populate a Redshift cluster with our data

In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

### Access credentials securely

In [2]:
load_dotenv(".env")

user = os.environ.get("REDSHIFT_USERNAME")
password = os.environ.get("REDSHIFT_PASSWORD")
host = os.environ.get("REDSHIFT_HOST")
iamrole = os.environ.get("IAM_role")

In [11]:
iamrole

'arn:aws:iam::888374473732:role/redshift-jupysql-test-read-s3-role'

### Form connection string

Note, this assumes we added

```bash
poetry add sqlalchemy-redshift

```

In [3]:
%pip install redshift-connector "sqlalchemy<2" --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
url = URL.create(
    drivername="redshift+redshift_connector",
    username=user,
    password=password,
    host=host,
    port=5439,
    database="dev",
)

engine = create_engine(url)

### Load JupySQL extension

In [5]:
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/automate-elt-github'

Settings changed:

Config,value
displaycon,False
feedback,True
autopandas,True
named_parameters,True


### Use the engine to initialize access to our Redshift via the alias `redshift`

In [6]:
%sql engine --alias redshift

### Add data to S3 bucket

In [88]:
! aws s3 cp expanded_data/account.csv s3://ploomber-redshift-data/nyc-taxi/account.csv


5020.66s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


upload: expanded_data/account.csv to s3://ploomber-redshift-data/nyc-taxi/account.csv


### Create table `ridership` from data in S3 bucket

Ensure you set up your access key and secret access keys!

In [93]:
%%sql
DROP TABLE IF EXISTS account CASCADE;

""


In [94]:
%%sql
CREATE TABLE IF NOT EXISTS account
(account_id INT,
 district_id INT,
 frequency VARCHAR(50),
 date VARCHAR(50));

 COPY account
FROM 's3://ploomber-redshift-data/nyc-taxi/account.csv'
IAM_ROLE '{{iamrole}}'
IGNOREHEADER 1
CSV;

""


In [95]:
result = %sql select * from account

result

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102
...,...,...,...,...
4495,124,55,POPLATEK MESICNE,971228
4496,3958,59,POPLATEK MESICNE,971228
4497,777,30,POPLATEK MESICNE,971228
4498,1573,63,POPLATEK MESICNE,971229
